# 랭체인 사전 준비

In [13]:
# 패키지 설치
!pip install langchain==0.0.181
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.37.1
    Uninstalling openai-1.37.1:
      Successfully uninstalled openai-1.37.1


In [1]:
# 환경변수 준비
import os
os.environ["OPENAI_API_KEY"] = 'API_KEY'

In [2]:
# 패키지 설치
!pip install tiktoken
!pip install faiss-gpu

# 프롬프트 템플릿 생성

In [3]:
from langchain.prompts import PromptTemplate

# 입력 변수가 없는 프롬프트 템플릿 만들기
no_input_prompt = PromptTemplate(
    input_variables=[],
    template="멋진 동물이라고 하면?"
)

# 프롬프트 생성
print(no_input_prompt.format())

멋진 동물이라고 하면?


In [4]:
from langchain.prompts import PromptTemplate

# 하나의 입력 변수가 있는 프롬프트 템플릿 만들기
one_input_prompt = PromptTemplate(
    input_variables=["content"],
    template="멋진 {content}이라고 하면?"
)

# 프롬프트 생성
print(one_input_prompt.format(content="동물"))

멋진 동물이라고 하면?


In [5]:
from langchain.prompts import PromptTemplate

# 여러 개의 입력 변수가 있는 프롬프트 템플릿 만들기
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="{adjective}{content}이라고 하면?"
)

# 프롬프트 생성
print(multiple_input_prompt.format(adjective="멋진", content="동물"))

멋진동물이라고 하면?


In [6]:
from langchain.prompts import PromptTemplate

# jinja2를 이용한 프롬프트 템플릿 준비
jinja2_prompt = PromptTemplate(
    input_variables=["items"],
    template_format="jinja2",
    template="""
{% for item in items %}
Q: {{ item.question }}
A: {{ item.answer }}
{% endfor %}
"""
)

# 프롬프트 생성
items=[
    {"question": "foo", "answer": "bar"},
    {"question": "1", "answer": "2"}
]
print(jinja2_prompt.format(items=items))



Q: foo
A: bar

Q: 1
A: 2



# 답변 예시가 포함된 프롬프트 템플릿

In [7]:
from langchain.prompts import FewShotPromptTemplate

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# 답변 예시를 포함한 프롬프트 템플릿 만들기
prompt_from_string_examples = FewShotPromptTemplate(
    examples=examples, # 답변 예시
    example_prompt=example_prompt, # 프롬프트 템플릿
    prefix="모든 입력에 대한 반의어를 입력하세요", # 접두사
    suffix="입력: {adjective}\n출력:", # 접미사
    input_variables=["adjective"], # 입력 변수
    example_separator="\n\n" # 구분 기호

)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 밝은
출력: 어두운

입력: 재미있는
출력: 지루한

입력: 큰
출력:


# LengthBasedExampleSelector

In [11]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
    {"input": "활기찬", "output": "무기력한"},
    {"input": "높은", "output": "낮은"},
    {"input": "빠른", "output": "느린"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# LengthBasedExampleSelector 생성
example_selector = LengthBasedExampleSelector(
    examples=examples, # 답변 예시
    example_prompt=example_prompt, # 프롬프트 템플릿
    max_length=10, # 문자열의 최대 길이
)

# FewShotPromptTemplate 생성
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,
    prefix="모든 입력에 대한 반의어를 입력하세요",
    suffix="입력: {adjective}\n출력:",
    input_variables=["adjective"],
    example_separator="\n\n"
)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 밝은
출력: 어두운

입력: 재미있는
출력: 지루한

입력: 큰
출력:


# SemanticSimilarityExampleSelector

In [12]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
    {"input": "활기찬", "output": "무기력한"},
    {"input": "높은", "output": "낮은"},
    {"input": "빠른", "output": "느린"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# SemanticSimilarityExampleSelector 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, # 답변 예시
    embeddings=OpenAIEmbeddings(), # 임베디드 생성 클래스
    vectorstore_cls=FAISS, # 임베디드 유사 검색 클래스
    k=3 # 답변 예시 개수
)

# FewShotPromptTemplate 생성
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,
    prefix="모든 입력에 대한 반의어를 입력하세요",
    suffix="입력: {adjective}\n출력:",
    input_variables=["adjective"],
    example_separator="\n\n"
)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 높은
출력: 낮은

입력: 재미있는
출력: 지루한

입력: 밝은
출력: 어두운

입력: 큰
출력:


# MaxMarginalRelevanceExampleSelector

In [13]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
    {"input": "활기찬", "output": "무기력한"},
    {"input": "높은", "output": "낮은"},
    {"input": "빠른", "output": "느린"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# MaxMarginalRelevanceExampleSelector 생성
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples, # 답변 예시
    embeddings=OpenAIEmbeddings(), # 임베디드 생성 클래스
    vectorstore_cls=FAISS, # 임베디드 유사 검색 클래스
    k=3 # 답변 예시 개수
)

# FewShotPromptTemplate 준비
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="모든 입력에 대한 반의어를 입력하세요",
    suffix="입력: {adjective}\n출력:",
    input_variables=["adjective"],
    example_separator="\n\n"

)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 높은
출력: 낮은

입력: 재미있는
출력: 지루한

입력: 밝은
출력: 어두운

입력: 큰
출력:
